In [1]:
!pip install transformers

You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [2]:
import pandas as pd

#headlines_df = pd.read_csv('../data/analyst_ratings_processed.csv')
#headlines_df = headlines_df.sample(300, random_state=42)

In [10]:
headlines_df.head()

,Unnamed: 0,title,date,stock
951347,963395.0,Top Stocks In The Surety & Title Insurance Ind...,2010-04-13 05:00:00-04:00,ORI
98223,102011.0,Asterias Provides 6 Mo. Data Readout From Its ...,2018-07-17 07:05:00-04:00,AST
940133,952152.0,ONEOK Partners to Participate in Bakken Day,2010-08-04 16:22:00-04:00,OKS
185334,189463.0,Shares of Broadsoft to Resume Trade at 4:35PM EST,2013-11-04 16:12:00-05:00,BSFT
524898,535387.0,"Courier Corp Holder Gamco Reports 6.97%, Up Fr...",2015-03-30 16:20:00-04:00,GBL


In [58]:
import numpy as np

headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

print(headlines_list[:5])

['Bank of America Reiterates Buy and PT of $49 on Discovery Communications', 'Primoris Wins $50M In Solar Awards', 'How The Toshiba-Western Digital Beef Could Be A Boon For Micron Technology', 'Teens Rank Corporations By Coolness; Do Rankings Correlate To Success?', "Hearing Barclays Says Northern Trust Outlook Is 'Slightly Mute' As CFO Guided Q3 Net Income Flat To Modestly Lower vs Q2"]


In [15]:
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')

In [17]:
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([300, 3])


In [18]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.9402, 0.0254, 0.0344],
        [0.2727, 0.6773, 0.0500],
        [0.0830, 0.7072, 0.2098],
        [0.9374, 0.0175, 0.0451],
        [0.4195, 0.0178, 0.5627],
        [0.8486, 0.0541, 0.0973],
        [0.0215, 0.2806, 0.6979],
        [0.1797, 0.0092, 0.8111],
        [0.8760, 0.0111, 0.1129],
        [0.7264, 0.0188, 0.2548],
        [0.0858, 0.0994, 0.8148],
        [0.0789, 0.0100, 0.9112],
        [0.7675, 0.0193, 0.2131],
        [0.0080, 0.9702, 0.0218],
        [0.9395, 0.0110, 0.0495],
        [0.8589, 0.1016, 0.0395],
        [0.7818, 0.1004, 0.1177],
        [0.0080, 0.9754, 0.0166],
        [0.0580, 0.2505, 0.6915],
        [0.2982, 0.4383, 0.2635],
        [0.0094, 0.9708, 0.0198],
        [0.9154, 0.0385, 0.0461],
        [0.0365, 0.0528, 0.9107],
        [0.0282, 0.6327, 0.3391],
        [0.0243, 0.1238, 0.8520],
        [0.0350, 0.0672, 0.8978],
        [0.0246, 0.6180, 0.3574],
        [0.0368, 0.1453, 0.8179],
        [0.0238, 0.1664, 0.8099],
        [0.009

In [20]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

In [43]:
df.sort_values(by='Neutral', ascending=False)

,Headline,Positive,Negative,Neutral
271,Annual Shareholder Meetings Being Held Today By: Fiserv and MGP Ingredients,0.023845,0.020296,0.955859
43,Sequential Brands Reports Launch of Office by Martha Stewart Exclusively at Staples,0.028870,0.019753,0.951377
250,"UBS Research Summary (FLEX, ARW, BHI, BKI, CTXS, DPS, EGN, ESRX, ISIL, LIS, PH, TMO, UHS, WLP, XLNX, NSC)",0.023946,0.026518,0.949535
77,"Amazon Is Taking Over The Market, One Sector At A Time",0.041424,0.012997,0.945579
109,"Benzinga’s Volume Movers (BARE, MAPP, HWCC, LEAP, IART)",0.039839,0.014831,0.945331
45,"Zacks Industry Outlook Highlights: UnitedHealth Group, Humana, HealthSpring and Universal American - Press Releases",0.029708,0.025361,0.944931
139,Shares of Broadsoft to Resume Trade at 4:35PM EST,0.040232,0.016843,0.942925
182,This ETF Is No Turkey,0.027637,0.030163,0.942199
58,Hearing Prescience Point Has Issued Short Idea On Kellogg; BZ NOTE: Firm Has Not Yet Tweeted On Stock,0.021328,0.036705,0.941967
161,"Benzinga's M&A Chatter for Friday September 28, 2012",0.021499,0.036781,0.941720


In [3]:
# Read JSON
import json
json_file_path = '../data/2021-11-16.json'

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [19]:
contents

{'TDOC': {'news': [['Nov-16-21',
    "Here's Why Investing for the Long Term Matters",
    'TDOC'],
   ['Nov-16-21',
    'What Might Big Tech Regulation Mean for Companies in Other Industries?',
    'TDOC'],
   ['Nov-16-21', '3 Companies With Stellar Acquisition Records', 'TDOC'],
   ['Nov-14-21',
    'Want $1 Million? Invest $250,000 in These 5 Stocks and Wait 10 Years',
    'TDOC'],
   ['Nov-13-21', 'Is Teladoc Stock a Smart Buy After Q3 Earnings?', 'TDOC'],
   ['Nov-12-21',
    "ARK Invest Stocks To Buy And Watch: 6 Stocks That Cathie Wood's ARK ETFs Own; Tesla Skids On Elon Musk Sale",
    'TDOC'],
   ['Nov-11-21', '11 Best Healthcare Stocks To Invest In', 'TDOC'],
   ['Nov-11-21',
    '3 Stocks Than Can Turn $10,000 Into $50,000 by 2025',
    'TDOC'],
   ['Nov-10-21',
    'Why Fastly, Teladoc Health, and MercadoLibre Stocks Fell Sharply Today',
    'TDOC'],
   ['Nov-10-21',
    'Was The Smart Money Right About Teladoc Health, Inc (TDOC)?',
    'TDOC'],
   ['Nov-08-21',
    'Got $3

In [4]:
new_ls = []
for ticker in contents.keys():
    for item in contents[ticker]['news']:
        item.append(ticker)
        new_ls.append(item)
        

In [5]:
round(len(new_ls)/300)

6

In [6]:
import numpy as np
headlines_array = np.array(new_ls)
np.random.shuffle(headlines_array)


In [16]:
headlines_array[0]

array(['Sep-27-21',
       'TotalEnergies and Safran Create a Strategic Partnership to Accelerate the Decarbonization of the Aviation Industry',
       'TTE'], dtype='<U223')

In [17]:
from tqdm import tqdm
import torch

df = pd.DataFrame(columns = ["date", "ticker","headline", "positive", "negative", "neutral"])

for group in tqdm(np.array_split(headlines_array, round(len(headlines_array)/300))):
    headlines_list = list(group[:,1])
    ticker_list = list(group[:,-1])
    date_list = list(group[:,0])

    inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()

    table = {'date':date_list,
        'ticker':ticker_list,
        'headline':headlines_list,
            "positive":positive,
            "negative":negative, 
            "neutral":neutral}
    
    df = df.append(pd.DataFrame.from_dict(table))
df = df.reset_index(drop=True)

100%|██████████| 6/6 [17:24<00:00, 174.02s/it]


In [18]:
df

,date,ticker,headline,positive,negative,neutral
0,Sep-27-21,TTE,TotalEnergies and Safran Create a Strategic Pa...,0.873221,0.008129,0.118650
1,Oct-26-21,TTE,"Plastic Recycling: Plastic Energy, Freepoint E...",0.661821,0.007918,0.330261
2,Oct-08-21,AMGN,3 High-Yielding Dividend Stocks Trading Near T...,0.018288,0.956064,0.025648
3,Nov-02-21,RDS-A,7 Oil Stocks That Could Be Portfolio Gushers a...,0.637146,0.275955,0.086899
4,Oct-04-21,ADBE,10 Quality Stocks Ready to Bounce Back After S...,0.908855,0.032371,0.058774
...,...,...,...,...,...,...
1682,Jul-11-21,CRSP,Intellia Therapeutics' Next Hurdles After Its ...,0.302343,0.023560,0.674097
1683,Nov-10-21,MSFT,"Valve Delays Steam Game Console, Citing Supply...",0.008933,0.940111,0.050956
1684,Nov-15-21,RDS-A,Shell shake-up leaves Dutch royally hacked off,0.026209,0.887695,0.086096
1685,Oct-20-21,ADBE,U.S. Online Holiday Sales Expected to Rise 10%...,0.910568,0.025059,0.064372


In [11]:
df.groupby('Ticker').mean().sort_values(by='Positive')

,Positive,Negative,Neutral
Ticker,,,
SBS,0.137380,0.047275,0.815345
RDS-A,0.195121,0.292276,0.512603
MO,0.210692,0.171228,0.618080
TDOC,0.217598,0.127380,0.655022
PLTR,0.231799,0.279699,0.488502
PYPL,0.234492,0.301713,0.463795
CRM,0.237562,0.137704,0.624734
V,0.260888,0.134954,0.604158
EURN,0.262495,0.105771,0.631734


In [5]:
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)


In [ ]:
print(outputs.logits.shape)

In [ ]:
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [ ]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Ticker':ticker_list,
    'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Ticker","Headline", "Positive", "Negative", "Neutral"])

In [ ]:
df